## Background

ADHD, also known as Attention-deficit/hyperactivity disorder (formerly also ADD) is a mental disorder that often goes undiagnosed in those it afflicts. It is also often mischaracterized as a disorder that 'just' makes kids rowdy. But it should be known that many adults are undiagnosed and it can greatly affect quality of life in all people.

Symptoms are characterized under two types:


### Inattentive Type

* Doesn’t pay close attention to details or makes careless mistakes in school or job tasks.
* Has problems staying focused on tasks or activities, such as during lectures, conversations or long reading.
* Does not seem to listen when spoken to (i.e., seems to be elsewhere).
* Does not follow through on instructions and doesn’t complete schoolwork, chores or job duties (may start tasks but quickly loses focus).
* Has problems organizing tasks and work (for instance, does not manage time well; has messy, disorganized work; misses deadlines).
* Avoids or dislikes tasks that require sustained mental effort, such as preparing reports and completing forms.
* Often loses things needed for tasks or daily life, such as school papers, books, keys, wallet, cell phone and eyeglasses.
* Is easily distracted.
* Forgets daily tasks, such as doing chores and running errands. Older teens and adults may forget to return phone calls, pay bills and keep appointments.


### Hyperactive/Impulsive Type

* Fidgets with or taps hands or feet, or squirms in seat.
* Not able to stay seated (in classroom, workplace).
* Runs about or climbs where it is inappropriate.
* Unable to play or do leisure activities quietly.
* Always “on the go,” as if driven by a motor.
* Talks too much.
* Blurts out an answer before a question has been finished (for instance may finish people’s sentences, can’t wait to speak in conversations).
* Has difficulty waiting for his or her turn, such as while waiting in line.
* Interrupts or intrudes on others (for instance, cuts into conversations, games or activities, or starts using other people’s things without permission). Older teens and adults may take over what others are doing.

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

C:\Users\goober\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
df = pd.read_csv('dataset/adhd_data')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/adhd_data'